In [1]:
import logging
import time
import lotr

logging.basicConfig(level=logging.INFO, format='%(asctime)s: %(message)s')

In [2]:
timestamp = time.time()
conf = lotr.read_conf()
lotr.clear_project_folders()
lotr.download_sheet(conf)
sets = lotr.get_sets(conf)

changes = False
for set_data in sets:
    set_id, set_name, set_row = set_data
    if 'octgn' in conf['outputs']:
        lotr.generate_octgn_xml(conf, set_name, set_row)

    for lang in conf['languages']:
        lotr.backup_previous_xml(conf, set_id, set_name, lang)
        lotr.generate_xml(conf, set_id, set_name, set_row, lang)
        lotr.update_xml(conf, set_id, set_name, lang)
        new_hash, old_hash = lotr.calculate_hashes(set_id, set_name, lang)
        if new_hash != old_hash:
            changes = True

        lotr.copy_raw_images(conf, set_id, set_name, lang)
        lotr.copy_xml(set_id, set_name, lang)

if changes:
    lotr.create_project()
else:
    logging.info('No changes since the last run, skipping creating the'
                 ' project')

logging.info('Done (%ss)', round(time.time() - timestamp, 3))

2020-08-27 19:52:49,734: Reading project configuration...
2020-08-27 19:52:49,744: ...Reading project configuration (0.006s)
2020-08-27 19:52:49,745: Clearing the project folders...
2020-08-27 19:52:49,747: ...Clearing the project folders (0.001s)
2020-08-27 19:52:49,748: Downloading cards spreadsheet from Google Drive...
2020-08-27 19:52:53,002: ...Downloading cards spreadsheet from Google Drive (3.253s)
2020-08-27 19:52:53,003: Getting all sets to work on...
2020-08-27 19:52:55,048: ...Getting all sets to work on (2.044s)
2020-08-27 19:52:55,059: [ALeP - Oaths of the Rohirrim] Generating set.xml file for OCTGN...
2020-08-27 19:52:58,407: [ALeP - Oaths of the Rohirrim] ...Generating set.xml file for OCTGN (3.347s)
2020-08-27 19:52:58,409: [ALeP - Oaths of the Rohirrim, English] Backing up a previous Strange Eons xml file...
2020-08-27 19:52:58,414: [ALeP - Oaths of the Rohirrim, English] ...Backing up a previous Strange Eons xml file (0.004s)
2020-08-27 19:52:58,416: [ALeP - Oaths of 

### Strange Eons Part:

1. Open `setGenerator.seproject` in Strange Eons.
2. Run `Script/makeCards` script by double clicking it.
3. Once completed, close Strange Eons (wait until it finished packing the project).

In [3]:
timestamp = time.time()
for set_data in sets:
    set_id, set_name, _ = set_data
    for lang in conf['languages']:
        skip_set, skip_ids = lotr.get_skip_info(set_id, set_name, lang)
        if skip_set:
            logging.info('[%s, %s] No changes since the last run,'
                         ' skipping', set_name, lang)
            continue

        if 'db_octgn' in conf['outputs']:
            lotr.generate_jpg300_nobleed(set_id, set_name, lang, skip_ids)
            if 'db' in conf['outputs']:
                lotr.generate_db(set_id, set_name, lang)
            if 'octgn' in conf['outputs']:
                lotr.generate_octgn(set_id, set_name, lang)

        if 'pdf' in conf['outputs']:
            lotr.generate_png300_pdf(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_pdf(set_id, set_name, lang)

        if 'makeplayingcards' in conf['outputs']:
            lotr.generate_png800_bleedmpc(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_mpc(conf, set_id, set_name, lang)

        if 'drivethrucards' in conf['outputs']:
            lotr.generate_jpg300_bleeddtc(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_dtc(conf, set_id, set_name, lang)

logging.info('Done (%ss)', round(time.time() - timestamp, 3))

2020-08-27 20:26:11,640: [ALeP - Oaths of the Rohirrim, English] Getting skip information...
2020-08-27 20:26:11,645: [ALeP - Oaths of the Rohirrim, English] ...Getting skip information (0.004s)
2020-08-27 20:26:11,646: [ALeP - Oaths of the Rohirrim, English] Generating images for DB and OCTGN outputs...
2020-08-27 20:26:11,808: [ALeP - Oaths of the Rohirrim, English] ...Generating images for DB and OCTGN outputs (0.161s)
2020-08-27 20:26:11,809: [ALeP - Oaths of the Rohirrim, English] Generating DB outputs...
2020-08-27 20:26:12,232: [ALeP - Oaths of the Rohirrim, English] ...Generating DB outputs (0.423s)
2020-08-27 20:26:12,233: [ALeP - Oaths of the Rohirrim, English] Generating OCTGN outputs...
2020-08-27 20:26:12,738: [ALeP - Oaths of the Rohirrim, English] ...Generating OCTGN outputs (0.505s)
2020-08-27 20:26:12,739: [ALeP - Oaths of the Rohirrim, English] Generating images for PDF outputs...
2020-08-27 20:26:18,530: [ALeP - Oaths of the Rohirrim, English] CompletedProcess(args='

Now there should be the following outputs:

- `Output/OCTGN/<set name>/`: `<octgn id>/set.xml` and `<set name>.<language>.o8c` image packs for OCTGN (300 dpi JPG).
- `Output/DB/<set name>.<language>/`: 300 dpi JPG images for general purposes.
- `Output/PDF/<set name>.<language>/`: PDF files in `A4` and `letter` format for home printing.
- `Output/DriveThruCards/<set name>.<language>/`: `zip` and `7z` archives of 300 dpi PNG images to be printed on DriveThruCards.com.
- `Output/MakePlayingCards/<set name>.<language>/`: `zip` and `7z` archives of 800 dpi PNG images to be printed on MakePlayingCards.com.